<a href="https://colab.research.google.com/github/vir-k01/CH5650/blob/main/CH5650_Unsupervised_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of crystals (in the form of 2D Diffraction fingerprints) using Unsupervised Learning
-Authored by Vir Karan

### Load the required packages and dataset

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans
from itertools import permutations
from multiprocessing import Pool
import pickle
np.random.seed(42)

I'm loading the dataset from my google drive, hence the below cell. Data can be downloaded from:

Pristine dataset (Diffraction fingerprints of crystals): https://dataverse.harvard.edu/api/access/datafile/3238702?format=original

Pristine dataset (Labels of crystals): https://dataverse.harvard.edu/api/access/datafile/3238704?format=original

Dataset info: https://dataverse.harvard.edu/api/access/datafile/3238706?format=original

Defected dataset (Diffraction fingerprints of defected crystals): https://dataverse.harvard.edu/api/access/datafile/3238702?format=original

Defected dataset (Labels of defected crystals): https://dataverse.harvard.edu/api/access/datafile/3238704?format=original

Dataset is in the form of pickled files, have to open them and convert to numpy arrays.

In [4]:
from google.colab import drive
drive.mount('/content/drive')
X = pickle.load(open('/content/drive/MyDrive/Acads/CH5650/pristine_dataset_x.pkl', 'rb'), encoding='latin1')
y = pickle.load(open('/content/drive/MyDrive/Acads/CH5650/pristine_dataset_y.pkl', 'rb'), encoding='latin1')
X_val = pickle.load(open('/content/drive/MyDrive/Acads/CH5650/pristine_dataset_x_val.pkl', 'rb'), encoding='latin1')
y_val = pickle.load(open('/content/drive/MyDrive/Acads/CH5650/pristine_dataset_y_val.pkl', 'rb'), encoding='latin1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Build and train a simple autoencoder (architecture and hyperparameters are given in the report). Then, use K-Means to cluster the reduced order representation of the dataset. In this case, we'll iterate over several possible dimensions for the bottleneck layer (as given in the list "dims"). This is because, unlike PCA, we cannot get a direct plot between captured features and number of components (dimensions) to choose the number of components to keep. So, the model that gives the best results will be kept.

In [ ]:
dims = [2, 4, 8, 32] # This is the size of our encoded representations

acc = []
perms = []

for encoding_dim in dims:
  # This is our input image
  input_img = keras.Input(shape=(64*64*3,))
  e1 = layers.Dense(128, activation='relu')(input_img)
  e2 = layers.Dense(64, activation='relu')(e1)
  # "encoded" is the encoded representation of the input
  encoded = layers.Dense(encoding_dim, activation='relu')(e2)
  d1 = layers.Dense(64, activation='relu')(encoded)
  d2 = layers.Dense(128, activation='relu')(d1)
  # "decoded" is the lossy reconstruction of the input
  decoded = layers.Dense(64*64*3, activation='tanh')(d2)

  # This model maps an input to its reconstruction
  autoencoder = keras.Model(input_img, decoded)
  encoder = keras.Model(input_img, encoded) #This model maps an input to its learnt reduced order representation
  #Compile and train the autoencoder
  autoencoder.compile(optimizer='adam', loss='mean_squared_error')
  autoencoder.fit(X.reshape(10517, -1), X.reshape(10517, -1), epochs=10, validation_split=0.05)

  #use K-means to cluster the reduced order representation of the dataset
  enc = encoder.predict(X.reshape(10517, -1))
  kmeans = KMeans(n_clusters=7, init='random')
  kmeans.fit(enc)

  #plot the results
  plt.scatter(enc[:, 0], enc[:, 1], c=kmeans.predict(enc), cmap='jet')
  plt.ylabel('Encoding Dim 2')
  plt.xlabel('Encoding Dim 1')
  plt.title('Actual data with labels')
  plt.colorbar()
  plt.show()

  plt.scatter(enc[:, 0], enc[:, 1], c=y, cmap='jet')
  plt.ylabel('Encoding Dim 2')
  plt.xlabel('Encoding Dim 1')
  plt.title('Data with labels predicted by KMeans')
  plt.colorbar()
  plt.show()

  #Next we can iterate over all possible permutations of the labels in the k-means clustering
  #and get the labels with best accuracy 
  indices = []

  def euklidean_distance(a, b):
      return np.sqrt(((a - b) ** 2).sum())

  for x in enc:
      distances = np.array([euklidean_distance(x, centroid) for centroid in kmeans.cluster_centers_])
      indices.append(np.argmin(distances))
      
      
  def validate(permutation):
      correct_predictions = 0
      for index, y in zip(indices, y_val):
          if y == permutation[index]:
              correct_predictions += 1
      return correct_predictions / len(y_val)
      

  def search_for_best_permutation(permutations):
      best_permutation = None
      highest_accuracy = 0
      for index, p in enumerate(permutations):
          accuracy = validate(p)
          if accuracy > highest_accuracy:
              best_permutation = p
              highest_accuracy = accuracy
      return highest_accuracy, best_permutation
    

  permutations = list(permutations(range(7)))

  slices = np.array(range(17))*(int(len(permutations)/16))
  p = []
  for index, item in enumerate(slices):
      try:
          p += [permutations[item:slices[index+1]]]
      except:
          pass
          
  with Pool(16) as pool:
      results = pool.map(search_for_best_permutation, p)

  best_permutation = None
  highest_accuracy = 0

  for accuracy, permutation in results:
      if accuracy > highest_accuracy:
          best_permutation = permutation
          highest_accuracy = accuracy
  acc.append(highest_accuracy)
  perms.append(best_permutation)
  print('Dim: '+ str(encoding_dim))
  print('Best_permutation: ' + str(best_permutation))
  print('Accuracy: '+ str(highest_accuracy))

Next, try the same procedure as above but with Linear PCA. Here, select the top 50 components to keep (based on scree plot).

In [ ]:
pca = PCA(100)
pca.fit(X.reshape(10517, -1))
X_ =  pca.transform(X.reshape(10517, -1))

kmeans = KMeans(n_clusters=7, init='random')
kmeans.fit(X_[:, :50])

indices = []

def euklidean_distance(a, b):
    return np.sqrt(((a - b) ** 2).sum())

for x in X_[:, :50]:
    distances = np.array([euklidean_distance(x, centroid) for centroid in kmeans.cluster_centers_])
    indices.append(np.argmin(distances))
    
    
def validate(permutation):
    correct_predictions = 0
    for index, y in zip(indices, y_val):
        if y == permutation[index]:
            correct_predictions += 1
    return correct_predictions / len(y_val)
    

def search_for_best_permutation(permutations):
    best_permutation = None
    highest_accuracy = 0
    for index, p in enumerate(permutations):
        accuracy = validate(p)
        if accuracy > highest_accuracy:
            best_permutation = p
            highest_accuracy = accuracy
    return highest_accuracy, best_permutation
  

permutations = list(permutations(range(7)))

slices = np.array(range(17))*(int(len(permutations)/16))
p = []
for index, item in enumerate(slices):
    try:
        p += [permutations[item:slices[index+1]]]
    except:
        pass
        
with Pool(16) as pool:
    results = pool.map(search_for_best_permutation, p)

best_permutation = None
highest_accuracy = 0

for accuracy, permutation in results:
    if accuracy > highest_accuracy:
        best_permutation = permutation
        highest_accuracy = accuracy

print('Best_permutation: ' + str(best_permutation))
print('Accuracy: '+ str(highest_accuracy))

out = kmeans.predict(X_[:, :50])
label_map = {0: best_permutation[0], 1: best_permutation[1], 2: best_permutation[2], 3: best_permutation[3], 4: best_permutation[4], 5: best_permutation[5], 6: best_permutation[6]}
lab = [label_map[e] for e in y]

plt.scatter(X_[:, 0], X_[:, 1], c=y, cmap='jet')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Data with actual labels')
plt.colorbar()
plt.show()


plt.scatter(X_[:, 0], X_[:, 1], c=out, cmap='jet')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Data with KMeans predicted labels')
plt.colorbar()
plt.show()

Finally, try out Kernel PCA (with a polynomial kernel) as well.

In [ ]:
kpca = KernelPCA(100, kernel='poly')
kpca.fit(X.reshape(10517, -1))
X_k =  kpca.transform(X.reshape(10517, -1))

kmeans = KMeans(n_clusters=7, init='random')
kmeans.fit(X_k[:, :50])

indices = []

def euklidean_distance(a, b):
    return np.sqrt(((a - b) ** 2).sum())

for x in X_k[:, :50]:
    distances = np.array([euklidean_distance(x, centroid) for centroid in kmeans.cluster_centers_])
    indices.append(np.argmin(distances))
    
    
def validate(permutation):
    correct_predictions = 0
    for index, y in zip(indices, y_val):
        if y == permutation[index]:
            correct_predictions += 1
    return correct_predictions / len(y_val)
    

def search_for_best_permutation(permutations):
    best_permutation = None
    highest_accuracy = 0
    for index, p in enumerate(permutations):
        accuracy = validate(p)
        if accuracy > highest_accuracy:
            best_permutation = p
            highest_accuracy = accuracy
    return highest_accuracy, best_permutation
  

permutations = list(permutations(range(7)))

slices = np.array(range(17))*(int(len(permutations)/16))
p = []
for index, item in enumerate(slices):
    try:
        p += [permutations[item:slices[index+1]]]
    except:
        pass
        
with Pool(16) as pool:
    results = pool.map(search_for_best_permutation, p)

best_permutation = None
highest_accuracy = 0

for accuracy, permutation in results:
    if accuracy > highest_accuracy:
        best_permutation = permutation
        highest_accuracy = accuracy

print('Best_permutation: ' + str(best_permutation))
print('Accuracy: '+ str(highest_accuracy))

out = kmeans.predict(X_[:, :50])
label_map = {0: best_permutation[0], 1: best_permutation[1], 2: best_permutation[2], 3: best_permutation[3], 4: best_permutation[4], 5: best_permutation[5], 6: best_permutation[6]}
lab = [label_map[e] for e in y]

plt.scatter(X_k[:, 0], X_k[:, 1], c=y, cmap='jet')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Data with actual labels')
plt.colorbar()
plt.show()


plt.scatter(X_k[:, 0], X_k[:, 1], c=out, cmap='jet')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Data with KMeans predicted labels')
plt.colorbar()
plt.show()